In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1K3QJmxvgc0_ZwXawSeV_oQll-6WeibaO", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/01_00_intro.mp3"))


In [ ]:
#@title 🎧 Code Walkthrough: Setup Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_01_setup_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Code Walkthrough: Implement Gaussian Score
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_05_implement_gaussian_score.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_02_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Manifold Hypothesis
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_03_manifold_hypothesis.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Gaussian Math
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_04_gaussian_math.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# Score Estimation and the Low-Density Problem -- Vizuara

## 1. Why Does This Matter?

Score-based generative models are at the heart of modern AI image generation. The **score function** -- the gradient of the log-density -- acts like a compass that points toward regions where data lives.

But this compass has a fatal flaw: it only works where data actually exists. In the vast empty spaces of high-dimensional data, the score becomes unreliable or meaningless. Understanding this problem is the first step toward understanding Noise Conditioned Score Networks (NCSN), which solved it elegantly.

By the end of this notebook, you will:
- Implement a score function from scratch for simple distributions
- Visualize where score estimation fails
- Understand the **manifold hypothesis** and why it matters for generative models
- See firsthand why we need multi-scale noise perturbation

## 2. Building Intuition

Let us start with a concrete analogy.

Imagine you have a compass that works perfectly when you are near a city. It always points you toward the nearest settlement. But the moment you step into a vast desert -- far from any civilization -- the compass starts spinning wildly. There is nothing nearby to point toward, so it gives you garbage directions.

This is exactly what happens with score functions in machine learning. The "cities" are clusters of data points, and the "desert" is the empty space between them.

**Key question:** If our data lives in a 784-dimensional space (like 28x28 images), how much of that space actually contains data?

The answer is: almost none. Data lives on a thin **manifold** -- a low-dimensional surface embedded in high-dimensional space. This is called the **manifold hypothesis**.

## 3. The Mathematics

The score function is defined as the gradient of the log probability density:

$$s(x) = \nabla_x \log p(x)$$

For a 1D Gaussian distribution with mean $\mu$ and standard deviation $\sigma$:

$$p(x) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)$$

Taking the log and then the gradient:

$$\log p(x) = -\frac{1}{2}\log(2\pi\sigma^2) - \frac{(x-\mu)^2}{2\sigma^2}$$

$$s(x) = \nabla_x \log p(x) = -\frac{x - \mu}{\sigma^2}$$

**Worked example:** Let $\mu = 0$, $\sigma = 1$.
- At $x = 0$: $s(0) = 0$ (at the peak, no direction to move)
- At $x = 1$: $s(1) = -1$ (pulled back toward center)
- At $x = -2$: $s(-2) = 2$ (pulled rightward toward center)
- At $x = 50$: $s(50) = -50$ (huge pull, but $p(50) \approx 0$!)

The score is mathematically defined at $x = 50$, but there is zero data there to learn it from. Any neural network trained with finite samples will produce garbage predictions at such points.

## 4. Let's Build It -- Component by Component

### 4.1 Implement the Gaussian Score Function

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

def gaussian_pdf(x, mu=0.0, sigma=1.0):
    """Compute Gaussian probability density."""
    return (1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)

def gaussian_score(x, mu=0.0, sigma=1.0):
    """Compute the score function for a Gaussian: s(x) = -(x - mu) / sigma^2."""
    return -(x - mu) / (sigma ** 2)

In [ ]:
#@title 🎧 What to Look For: Visualize Gaussian Score
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_06_visualize_gaussian_score.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Visualize the Score Function

In [ ]:
x = np.linspace(-5, 5, 200)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: PDF
axes[0].plot(x, gaussian_pdf(x), 'b-', linewidth=2)
axes[0].set_title('Gaussian PDF: p(x)')
axes[0].set_xlabel('x')
axes[0].set_ylabel('p(x)')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='k', linewidth=0.5)

# Plot 2: Score function
scores = gaussian_score(x)
axes[1].plot(x, scores, 'r-', linewidth=2)
axes[1].set_title('Score Function: s(x) = d/dx log p(x)')
axes[1].set_xlabel('x')
axes[1].set_ylabel('s(x)')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='k', linewidth=0.5)
axes[1].axvline(x=0, color='k', linewidth=0.5, linestyle='--', label='Mean')
axes[1].legend()

plt.tight_layout()
plt.show()

print("At x=0 (peak): score =", gaussian_score(0))
print("At x=2 (tail): score =", gaussian_score(2))
print("At x=50 (far away): score =", gaussian_score(50), "but p(50) =", gaussian_pdf(50))

In [ ]:
#@title 🎧 Listen: Mixture Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_08_mixture_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


**Checkpoint:** You should see that the score is zero at the mean and grows linearly in magnitude as you move away. The key insight: at $x = 50$, the score says "move left with force 50," but the probability there is essentially zero.

### 4.3 Mixture of Gaussians -- Where the Problem Gets Real

In [ ]:
#@title 🎧 Code Walkthrough: Mixture Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_09_mixture_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def mixture_pdf(x, centers=[-3.0, 3.0], sigma=0.5, weights=[0.5, 0.5]):
    """PDF of a mixture of Gaussians."""
    pdf = np.zeros_like(x)
    for c, w in zip(centers, weights):
        pdf += w * gaussian_pdf(x, mu=c, sigma=sigma)
    return pdf

def mixture_score_analytical(x, centers=[-3.0, 3.0], sigma=0.5, weights=[0.5, 0.5]):
    """Analytical score for mixture of Gaussians."""
    # s(x) = (d/dx p(x)) / p(x)
    pdf = mixture_pdf(x, centers, sigma, weights)
    dpdf = np.zeros_like(x)
    for c, w in zip(centers, weights):
        component_pdf = gaussian_pdf(x, mu=c, sigma=sigma)
        dpdf += w * component_pdf * (-(x - c) / sigma**2)
    # Avoid division by zero
    score = np.where(pdf > 1e-30, dpdf / pdf, 0.0)
    return score

In [ ]:
#@title 🎧 What to Look For: Visualize Mixture Score
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_10_visualize_mixture_score.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
x = np.linspace(-8, 8, 500)
pdf = mixture_pdf(x)
score = mixture_score_analytical(x)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PDF
axes[0].plot(x, pdf, 'b-', linewidth=2)
axes[0].fill_between(x, pdf, alpha=0.2)
axes[0].set_title('Mixture of Gaussians PDF')
axes[0].set_xlabel('x')
axes[0].set_ylabel('p(x)')
axes[0].grid(True, alpha=0.3)

# Score with reliability coloring
colors = np.where(pdf > 0.01, 'green', 'red')
axes[1].scatter(x[::5], score[::5], c=colors[::5], s=10, alpha=0.7)
axes[1].set_title('Score Function (Green=Reliable, Red=Unreliable)')
axes[1].set_xlabel('x')
axes[1].set_ylabel('s(x)')
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(-20, 20)

plt.tight_layout()
plt.show()

print("Green points: regions where p(x) > 0.01 (score is reliable)")
print("Red points: regions where p(x) < 0.01 (score is unreliable)")

In [ ]:
#@title 🎧 Before You Start: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_12_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


**Checkpoint:** The score function should be well-behaved (green) near the two Gaussian centers and chaotic/unreliable (red) far from them.

## 5. Your Turn

### TODO 1: Implement a 2D Score Function and Vector Field

In [ ]:
def score_2d_mixture(X, Y, centers=[(-3, 0), (3, 0)], sigma=0.5):
    """
    Compute the analytical score field for a 2D mixture of Gaussians.

    Args:
        X, Y: meshgrid arrays of shape (H, W)
        centers: list of (x, y) center coordinates
        sigma: standard deviation of each component

    Returns:
        score_x, score_y: score field components of shape (H, W)

    Hints:
        - For each point (x, y), compute the pdf contribution from each center
        - The score for a 2D Gaussian centered at (cx, cy) is:
          score_x = -(x - cx) / sigma^2, score_y = -(y - cy) / sigma^2
        - Weight by the relative pdf contribution of each component
    """
    # YOUR CODE HERE
    # Step 1: Compute the pdf from each component at every grid point
    # Step 2: Compute total pdf (sum of components)
    # Step 3: For each component, compute its score contribution weighted by pdf
    # Step 4: Divide by total pdf to get the mixture score
    pass

# Test your implementation
X, Y = np.meshgrid(np.linspace(-6, 6, 20), np.linspace(-4, 4, 15))
score_x, score_y = score_2d_mixture(X, Y)

plt.figure(figsize=(10, 6))
plt.quiver(X, Y, score_x, score_y, color='blue', alpha=0.7)
plt.scatter([-3, 3], [0, 0], c='red', s=100, zorder=5, label='Data centers')
plt.title('2D Score Field for Mixture of Gaussians')
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
#@title 🎧 Before You Start: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_14_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Demonstrate the Low-Density Problem

In [ ]:
def train_score_network_1d(data, n_epochs=2000, hidden_dim=64, lr=1e-3):
    """
    Train a simple neural network to estimate the score function from data.

    Args:
        data: 1D training data tensor of shape (N,)
        n_epochs: number of training epochs
        hidden_dim: hidden layer size
        lr: learning rate

    Returns:
        Trained model

    Hints:
        - Use denoising score matching: add noise, predict score of noisy distribution
        - Score of noisy distribution at sigma: -(x_noisy - x_clean) / sigma^2
        - Use a small fixed sigma (e.g., 0.1) for now
        - Model: Linear(1, hidden) -> SiLU -> Linear(hidden, hidden) -> SiLU -> Linear(hidden, 1)
    """
    # YOUR CODE HERE
    pass

# Generate mixture data and train
# Then compare learned score vs analytical score
# Especially look at low-density regions!

In [ ]:
#@title 🎧 Transition: Putting It Together Transition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_16_putting_it_together_transition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

In [ ]:
#@title 🎧 What to Look For: Full Picture Visualization
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_17_full_picture_visualization.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Let's see the full picture: data, PDF, analytical score, and the problem

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

x = np.linspace(-8, 8, 500)
pdf = mixture_pdf(x)

# Panel 1: Data histogram
data_samples = np.concatenate([
    np.random.normal(-3, 0.5, 2500),
    np.random.normal(3, 0.5, 2500)
])
axes[0, 0].hist(data_samples, bins=80, density=True, alpha=0.5, color='blue')
axes[0, 0].plot(x, pdf, 'r-', linewidth=2)
axes[0, 0].set_title('Data Distribution')
axes[0, 0].set_xlabel('x')

# Panel 2: Log probability (shows the problem!)
log_pdf = np.log(pdf + 1e-30)
axes[0, 1].plot(x, log_pdf, 'b-', linewidth=2)
axes[0, 1].axhline(y=-30, color='r', linestyle='--', label='log(p) -> -inf')
axes[0, 1].set_title('Log Probability (drops to -inf in gaps)')
axes[0, 1].set_xlabel('x')
axes[0, 1].set_ylabel('log p(x)')
axes[0, 1].legend()

# Panel 3: Score function
score = mixture_score_analytical(x)
axes[1, 0].plot(x, score, 'g-', linewidth=2)
axes[1, 0].set_title('Score Function')
axes[1, 0].set_xlabel('x')
axes[1, 0].set_ylabel('s(x)')
axes[1, 0].set_ylim(-30, 30)

# Panel 4: Score reliability map
reliability = pdf / pdf.max()
scatter = axes[1, 1].scatter(x, score, c=reliability, cmap='RdYlGn', s=5, vmin=0, vmax=0.3)
axes[1, 1].set_title('Score Colored by Reliability')
axes[1, 1].set_xlabel('x')
axes[1, 1].set_ylabel('s(x)')
axes[1, 1].set_ylim(-30, 30)
plt.colorbar(scatter, ax=axes[1, 1], label='Relative density')

plt.tight_layout()
plt.show()

print("\nKey insight: the score function is well-defined everywhere,")
print("but only RELIABLE where the data density is significant (green regions).")
print("In the red regions, no training data exists to learn the correct score.")

In [ ]:
#@title 🎧 Transition: Training Results Transition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_19_training_results_transition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Training and Results

In [ ]:
#@title 🎧 Code Walkthrough: Train Score Net
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_20_train_score_net.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Let's train a neural network to learn the score and see where it fails

import torch
import torch.nn as nn

class SimpleScoreNet(nn.Module):
    def __init__(self, hidden=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, hidden),
            nn.SiLU(),
            nn.Linear(hidden, hidden),
            nn.SiLU(),
            nn.Linear(hidden, 1),
        )

    def forward(self, x):
        return self.net(x)

# Generate training data
data = torch.cat([
    torch.randn(2500, 1) * 0.5 - 3,
    torch.randn(2500, 1) * 0.5 + 3,
])

# Train with denoising score matching (single sigma)
model = SimpleScoreNet()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
sigma_noise = 0.3  # Fixed small noise level

losses = []
for epoch in range(3000):
    noise = torch.randn_like(data)
    noisy = data + sigma_noise * noise
    target = -noise / sigma_noise  # DSM target

    pred = model(noisy)
    loss = ((pred - target) ** 2).mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

print(f"Final loss: {losses[-1]:.4f}")

In [ ]:
#@title 🎧 What to Look For: Compare Learned Analytical
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_21_compare_learned_analytical.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Compare learned score vs analytical score
x_test = np.linspace(-8, 8, 500)
x_tensor = torch.tensor(x_test, dtype=torch.float32).unsqueeze(1)

with torch.no_grad():
    learned_score = model(x_tensor).numpy().squeeze()

analytical = mixture_score_analytical(x_test)
pdf_test = mixture_pdf(x_test)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: full comparison
axes[0].plot(x_test, analytical, 'b-', linewidth=2, label='Analytical score', alpha=0.7)
axes[0].plot(x_test, learned_score, 'r--', linewidth=2, label='Learned score', alpha=0.7)
axes[0].set_title('Score Function: Analytical vs Learned')
axes[0].set_xlabel('x')
axes[0].set_ylabel('s(x)')
axes[0].set_ylim(-30, 30)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Right: error map
error = np.abs(learned_score - analytical)
axes[1].semilogy(x_test, error + 1e-6, 'k-', alpha=0.5, label='|error|')
ax2 = axes[1].twinx()
ax2.fill_between(x_test, pdf_test, alpha=0.2, color='blue', label='p(x)')
axes[1].set_title('Error (note: huge errors in low-density regions)')
axes[1].set_xlabel('x')
axes[1].set_ylabel('|Learned - Analytical|')
ax2.set_ylabel('p(x)')
axes[1].legend(loc='upper left')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

print("\nNotice: the learned score matches well near x=-3 and x=3 (where data exists)")
print("But diverges wildly in the gap around x=0 and at the tails!")
print("This is the LOW-DENSITY PROBLEM that NCSN solves.")

In [ ]:
#@title 🎧 Transition: Final Output Transition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_23_final_output_transition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

The key visualization from this notebook: a clear demonstration that score estimation fails in regions where the data density is near zero.

In [ ]:
#@title 🎧 What to Look For: Final Visualization
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_24_final_visualization.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Final summary visualization
fig, ax = plt.subplots(figsize=(12, 5))

# Background: color by reliability
for i in range(len(x_test) - 1):
    color = 'lightgreen' if pdf_test[i] > 0.01 else 'lightsalmon'
    ax.axvspan(x_test[i], x_test[i+1], alpha=0.3, color=color)

ax.plot(x_test, analytical, 'b-', linewidth=2.5, label='True score')
ax.plot(x_test, learned_score, 'r--', linewidth=2.5, label='Learned score')
ax.set_title('The Low-Density Problem: Score Estimation Fails in Empty Regions',
             fontsize=13, fontweight='bold')
ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('Score s(x)', fontsize=12)
ax.set_ylim(-25, 25)
ax.legend(fontsize=11)
ax.annotate('Reliable\n(data here)', xy=(-3, -15), fontsize=10, color='green',
            ha='center', fontweight='bold')
ax.annotate('Reliable\n(data here)', xy=(3, -15), fontsize=10, color='green',
            ha='center', fontweight='bold')
ax.annotate('UNRELIABLE\n(no data)', xy=(0, 18), fontsize=11, color='red',
            ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nThis is the fundamental problem that Noise Conditioned Score Networks solve.")
print("In the next notebook, we will see how adding noise at multiple scales")
print("fills in these empty regions and makes score estimation reliable everywhere.")

In [ ]:
#@title 🎧 Listen: Reflection Next Steps
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_25_reflection_next_steps.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

**What we learned:**
1. The score function $s(x) = \nabla_x \log p(x)$ points toward high-density regions
2. For mixture distributions, the score is only reliable where data actually exists
3. In high-dimensional spaces, most of the space is empty (manifold hypothesis)
4. A neural network trained with finite data cannot learn the score in empty regions

**Reflection questions:**
- Why does the manifold hypothesis make the problem worse in higher dimensions?
- If we could somehow make $p(x) > 0$ everywhere, would the problem go away?
- What happens to the score estimate if we add a small amount of Gaussian noise to all our data points?

**Next notebook:** We will see how adding noise at multiple scales elegantly solves this problem, leading to the Noise Conditioned Score Network.